In [1]:
!pip install albumentations --user

# Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from unet import build_unet
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.metrics import Recall, Precision
import keras.backend as K
from albumentations import HorizontalFlip, VerticalFlip, Rotate

# Global Parameters

In [2]:
H = 512
W = 256
model_name = f"model_segmentation_{W}x{H}px.h5"

# Define Some Functions

In [3]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path, split=0.2):
    images = sorted(glob(f"{path}/*/tumor/images/*"))
    masks = sorted(glob(f"{path}/*/tumor/masks/*"))

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def load_trainable_data(path):
    images = sorted(glob(f"{path}/images/*.jpg"))
    masks = sorted(glob(f"{path}/masks/*.jpg"))
    
    return (images,masks)

def augment_data(images, masks, save_path, augment=True):
    """ Performing data augmentation. """
    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting the dir name and image name """
        dir_name = x.split("\\")[-3]
        name = dir_name + "_" + x.split("\\")[-1].split(".")[0]

        """ Read the image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        if augment == True:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = Rotate(limit=45, p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]

        else:
            X = [x]
            #print(len(X))
            Y =[y]

        idx = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))
            m = m/255.0
            m = (m > 0.5) * 255

            if len(X) == 1:  # each loop it will take one image 
                tmp_image_name = f"{name}.jpg"
                tmp_mask_name  = f"{name}.jpg"
            else:
                tmp_image_name = f"{name}_{idx}.jpg"
                tmp_mask_name  = f"{name}_{idx}.jpg"

            image_path = os.path.join(save_path, "images/", tmp_image_name)
            mask_path  = os.path.join(save_path, "masks/", tmp_mask_name)
            
            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

            idx += 1


def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (h, w)
    x = cv2.resize(x, (W, H))   ## (h, w)
    x = x / 255.0               ## (h, w)
    x = x.astype(np.float32)    ## (h, w)
    x = np.expand_dims(x, axis=-1)## (h, w, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


def save_results(image, mask, y_pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred * 255

    line = np.ones((H, 10, 3)) * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

# Set hyperparameters

In [4]:
create_dir("files")

batch_size = 32
lr = 1e-4
num_epochs = 100
model_path = os.path.join("files", model_name)
csv_path = os.path.join("files", "log.csv")


# load a Dataset

In [13]:
dataset_path = "Dataset/"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test : {len(test_x)} - {len(test_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

Train: 1384 - 1384
Valid: 461 - 461
Test : 461 - 461


## make Data Augmentation for training data

In [13]:
create_dir("Dataset_new/images/")
create_dir("Dataset_new/masks/")

augment_data(train_x, train_y, "Dataset_new/", augment=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1384/1384 [01:07<00:00, 20.61it/s]


## load trainable data

In [14]:
train_x,train_y = load_trainable_data("Dataset_new/")
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)

In [7]:
len(train_x)

1384

# Model

In [16]:
model = build_unet((H, W, 3))
metrics = [dice_coef, iou, Recall(), Precision()]
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]


Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 512, 256, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 512, 256, 64)         0         ['batch_normalization[0][0]

 chNormalization)                                                                                 
                                                                                                  
 activation_8 (Activation)   (None, 32, 16, 1024)         0         ['batch_normalization_8[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_9 (Conv2D)           (None, 32, 16, 1024)         9438208   ['activation_8[0][0]']        
                                                                                                  
 batch_normalization_9 (Bat  (None, 32, 16, 1024)         4096      ['conv2d_9[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activatio

 )                                                                   'activation_1[0][0]']        
                                                                                                  
 conv2d_16 (Conv2D)          (None, 512, 256, 64)         73792     ['concatenate_3[0][0]']       
                                                                                                  
 batch_normalization_16 (Ba  (None, 512, 256, 64)         256       ['conv2d_16[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_16 (Activation)  (None, 512, 256, 64)         0         ['batch_normalization_16[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_17

In [ ]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    callbacks=callbacks,
    shuffle=True
)

Epoch 1/100
312/312 [==============================] - ETA: 0s - loss: 0.6176 - dice_coef: 0.3824 - iou: 0.2547 - recall: 0.7912 - precision: 0.4160  
Epoch 1: val_loss improved from inf to 0.88856, saving model to files\model_segmentation_256x512px.h5


C:\Users\AI\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


312/312 [==============================] - 24713s 79s/step - loss: 0.6176 - dice_coef: 0.3824 - iou: 0.2547 - recall: 0.7912 - precision: 0.4160 - val_loss: 0.8886 - val_dice_coef: 0.1113 - val_iou: 0.0590 - val_recall: 1.0000 - val_precision: 0.0632 - lr: 1.0000e-04
Epoch 2/100
312/312 [==============================] - ETA: 0s - loss: 0.4874 - dice_coef: 0.5126 - iou: 0.3665 - recall: 0.7383 - precision: 0.5965  
Epoch 2: val_loss improved from 0.88856 to 0.75721, saving model to files\model_segmentation_256x512px.h5
312/312 [==============================] - 24341s 78s/step - loss: 0.4874 - dice_coef: 0.5126 - iou: 0.3665 - recall: 0.7383 - precision: 0.5965 - val_loss: 0.7572 - val_dice_coef: 0.2429 - val_iou: 0.1390 - val_recall: 0.1818 - val_precision: 0.7787 - lr: 1.0000e-04
Epoch 3/100
 66/312 [=====>........................] - ETA: 5:14:14 - loss: 0.4584 - dice_coef: 0.5416 - iou: 0.3957 - recall: 0.6764 - precision: 0.6024

# load model after training

In [6]:
with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss,"iou":iou}):
    model = tf.keras.models.load_model(os.path.join("files", model_name))

# Prediction and Evaluation

In [24]:
create_dir("results")
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_y)):
    """ Extracting the name """
    name = x.split("/")[-1]

    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## [H, w, 3]
    image = cv2.resize(image, (W, H))       ## [H, w, 3]
    x = image/255.0                         ## [H, w, 3]
    x = np.expand_dims(x, axis=0)           ## [1, H, w, 3]

    """ Reading the mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (W, H))

    """ Prediction """
    y_pred = model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)

    """ Saving the prediction """
    save_image_path = os.path.join("results", name)
    #save_results(image, mask, y_pred, save_image_path)
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    y_pred = y_pred * 255

    line = np.ones((H, 10, 3)) * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

    """ Flatten the array """
    mask = mask/255.0
    mask = (mask > 0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    SCORE.append([name, f1_value, jac_value, recall_value, precision_value])

100%|████████████████████████████████████████████████████████████████████████████████| 461/461 [20:55<00:00,  2.72s/it]


In [21]:
""" Metrics values """
score = [s[1:]for s in SCORE]
score = np.mean(score, axis=0)
print(f"F1: {score[0]:0.5f}")
print(f"Jaccard: {score[1]:0.5f}")
print(f"Recall: {score[2]:0.5f}")
print(f"Precision: {score[3]:0.5f}")

df = pd.DataFrame(SCORE, columns=["Image", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score.csv")

F1: 0.49530
Jaccard: 0.39058
Recall: 0.55051
Precision: 0.63221


In [14]:
x = "Dataset/P011/tumor/images/IMG-0003-00283.jpg"
y = "Dataset/P011/tumor/masks/Mask283.tif"
name = "3.png"
save_image_path = os.path.join("results", name)
"""Reading the Image"""
image = cv2.imread(x, cv2.IMREAD_COLOR) ## [H, w, 3]
image = cv2.resize(image, (W, H))       ## [H, w, 3]
x = image/255.0                         ## [H, w, 3]
x = np.expand_dims(x, axis=0)

""" Reading the mask """
mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
mask = cv2.resize(mask, (W, H))


mask = np.expand_dims(mask, axis=-1)
mask = np.concatenate([mask, mask, mask], axis=-1)
""" Prediction """
y_pred = model.predict(x, verbose=0)[0]
y_pred = np.squeeze(y_pred, axis=-1)
y_pred = y_pred >= 0.5
y_pred = y_pred.astype(np.int32)

y_pred = np.expand_dims(y_pred, axis=-1)
y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
y_pred = y_pred * 255

line = np.ones((H, 10, 3)) * 255

cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
cv2.imwrite(save_image_path, cat_images)

True